<a href="https://colab.research.google.com/github/NaveenOburi/Track-And-Consult-Healthcare-services/blob/master/Assign__1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [0]:
import numpy as np

In [8]:
dataset=pd.read_csv('/content/housing.csv')
dataset=dataset.dropna()
# Print the first ten records of the dataset.
dataset.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [0]:
y=dataset['median_house_value']
x=dataset.loc[:,'longitude':'median_income']

In [0]:
#Test size 0.3 means test sample will be 30% and training will be 70%. 
#Thus, ratio of train:test is 70:30
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=2003)

In [0]:
x_train_np=x_train.to_numpy()
y_train_np=y_train.to_numpy()

x_test_np=x_test.to_numpy()
y_test_np=y_test.to_numpy()

In [0]:
import torch

In [0]:
from torch.nn import Conv1d

In [0]:
from torch.nn import MaxPool1d

In [0]:
from torch.nn import Flatten

In [0]:
from torch.nn import Linear

In [0]:
from torch.nn.functional import relu

In [0]:
from torch.utils.data import DataLoader,TensorDataset

In [0]:
class cnnRegressor(torch.nn.Module):
  def __init__(self,batch_size,inputs,outputs):

    super(cnnRegressor, self).__init__()
    self.batch_size=batch_size
    self.inputs=inputs
    self.outputs=outputs
    self.input_layer=Conv1d(inputs, batch_size, 1)
    self.max_pooling_layer=MaxPool1d(1)
    self.conv_layer=Conv1d(batch_size,128,1)
    self.flatten_layer=Flatten()
    self.linear_layer=Linear(128,64)
    self.output_layer=Linear(64,outputs)
  def feed(self, input):
     input = input.reshape(self.batch_size, self.inputs, 1)
     output = relu(self.input_layer(input))

     output = relu(self.max_pooling_layer(output))
     output= relu(self.conv_layer(output))
     output = self.flatten_layer(output)
     output = self.linear_layer(output)
     output = self.output_layer(output)
     return output


In [20]:
from torch.optim import Adam
from torch.nn import L1Loss
!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score

     |████████████████████████████████| 112kB 7.0MB/s 


In [0]:
batch_size = 64
model =cnnRegressor(batch_size, x.shape[1], 1)
model.cuda()


# saving the model
torch.save(model.state_dict(), "student_id:(0873122)1dconv reg.model")


In [0]:
def model_loss(model, dataset, train = False, optimizer = None):

  performance = L1Loss()

  score_metric = R2Score()

  avg_loss = 0

  avg_score = 0

  count = 0

  for input, output in iter(dataset):

    predictions = model.feed(input)

    loss = performance(predictions, output)

    score_metric.update([predictions, output])
    score = score_metric.compute()

    if(train):

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

    avg_loss += loss.item()

    avg_score += score

    count += 1

  return avg_loss / count, avg_score/ count
  
  print(sum(p.numel() for p in model.parameters()))

In [25]:
# Define the number of epochs to train for 

epochs = 100

# Define the performance measure and optimizer Import the Adam (stochastic gradient descent) package from pytorch for
# our optimizer
optimizer = Adam(model.parameters(), lr = 1e-3)

# Convert the training set into torch variables for our model using the GPU
# as floats. The reshape is to remove a warning pytorch outputs otherwise.
inputs = torch.from_numpy(x_train_np).cuda().float()
outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0], 1)).cuda().float()

# Create a DataLoader instance to work with our batches
tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

# Start the training loop
for epoch in range(epochs):
  # Cycle through the batches and get the average loss
  avg_loss, avg_r2_score = model_loss (model, loader, train = True, optimizer = optimizer)

# Output the average loss
print("Epoch " + str(epoch + 1) + ":\n\tLoss = " + str(avg_loss) + "\n\tR^2 Score = " + str(avg_r2_score))

 


Epoch 100:
	Loss = 48354.14637051569
	R^2 Score = 0.6683872190955844


In [0]:
inputs = torch.from_numpy(x_test_np).cuda().float()
outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0], 1)).cuda().float()

tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle = True, drop_last=True)
 

avg_loss, avg_r2_score = model_loss(model, loader)
print("The model's L1 loss is: " + str(avg_loss))
print("The model's R^2 score is: " + str(avg_r2_score))

In [0]:
columns = dataset.columns.drop(['longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income'])

x= range(0, dataset.shape[0])
fig, ax = plt.subplots()

for column in dataset:
  ax.plot(x, dataset[column],label=column)

dataset.plot(subplots=True)

ax.set_title('housing Dataset')



In [0]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
z = x + y
end.record()

# Waits for everything to finish running
torch.cuda.synchronize()

print(start.elapsed_time(end))